In [ ]:
import sisl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Example of the $k$-point sampling for TBtrans.

In [ ]:
chain = sisl.Geometry([0]*3, sisl.Atom(1, R=1.), sc=[1, 1, 10])
chain.set_nsc([3, 3, 1])

# Transport along y-direction
chain = chain.tile(20, 0)
He = sisl.Hamiltonian(chain)
He.construct(([0.1, 1.1], [0, -1]))
Hd = He.tile(20, 1)
He.write('ELEC.nc')
Hd.write('DEVICE.nc')

In [ ]:
with open('RUN.fdf', 'w') as f:
    f.write("""
TBT.k [ 3 1 1 ]

TBT.DOS.A
TBT.Current.Orb

TBT.HS DEVICE.nc
%block TBT.Elec.Left
  HS ELEC.nc
  semi-inf-direction -a2
  electrode-position 1
%endblock
%block TBT.Elec.Right
  HS ELEC.nc
  semi-inf-direction +a2
  electrode-position end -1
%endblock
    """)

Run these two executables:

    tbtrans -D TRS RUN.fdf
    tbtrans -D NO_TRS -fdf TBT.Symmetry.TimeReversal:f RUN.fdf

In [ ]:
trs = sisl.get_sile('TRS/siesta.TBT.nc')
no_trs = sisl.get_sile('NO_TRS/siesta.TBT.nc')

In [ ]:
def plot_bond(tbt, E):
    xy = tbt.geometry.xyz[:, :2]
    vector = tbt.vector_current(0, E)[:, :2]
    atom = tbt.atom_current(0, E)
    # Normalize atomic current
    atom += 1
    atom *= 10 / atom.max()
    plt.scatter(xy[:, 0], xy[:, 1], atom);
    plt.quiver(xy[:, 0], xy[:, 1], vector[:, 0], vector[:, 1]);
    plt.gca().get_xaxis().set_visible(False)
    plt.gca().get_yaxis().set_visible(False)
    plt.tight_layout()

In [ ]:
plot_bond(trs, 1.)
plt.savefig('fig/trs.png')

In [ ]:
plot_bond(no_trs, 1.)
plt.savefig('fig/no_trs.png')